<a href="https://colab.research.google.com/github/mancudEOM/Ensemble-deep-learning/blob/main/testdatacafegn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
  from google.colab import drive
  drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
!ls

drive  sample_data


In [11]:
  import sys
  root_path = '/content/drive/My Drive/Groundnut_Leaf_dataset/' # This is the path to where in google drive the code is stored!
  sys.path.append(root_path)

In [12]:
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, Average
from keras.models import Model
from keras.layers import Input, Dense
import tensorflow as tf

In [13]:
datasetFolderName=root_path+'dataset'
MODEL_FILENAME=root_path+"model_cv.h5"
sourceFiles=[]
classLabels=['early_leaf_spot_1','early_rust_1','healthy_leaf_1',
              'late_leaf_spot_1','nutrition_deficiency_1','rust_1']
X=[]
Y=[]

img_rows, img_cols =  224, 224
test_path=datasetFolderName+'/test/'+'/rust_1/'
#+'Lot_2 (250).JPG'

In [14]:
class WeightedAverageLayer(tf.keras.layers.Layer):
    def __init__(self, w1, w2, w3, **kwargs):
        super(WeightedAverageLayer, self).__init__(**kwargs)
        self.w1 = w1
        self.w2 = w2
        self.w3 = w3

    def call(self, inputs):
        return self.w1 * inputs[0] + self.w2 * inputs[1] + self.w3 * inputs[2]

In [15]:


# Disable scientific notation for clarity
#np.set_printoptions(suppress=True)
# Load the model
model_1 =load_model(root_path+"model_5fold_MobileNetV2_GN07mm.h5",
                    compile=False)
model_1 = Model(inputs=model_1.inputs,
                outputs=model_1.outputs,
                name='model_5fold_MobileNetV2_GN07mm')
model_2 = load_model(root_path+"model_5fold_ResNet50V2_GN07.h5",
                     compile=False)
model_2 = Model(inputs=model_1.inputs,
                outputs=model_1.outputs,
                name='model_5fold_ResNet50V2_GN07')
model_3 = load_model(root_path+"model_5fold_InceptionV3_GN07mm.h5",
                     compile=False)
model_3 = Model(inputs=model_1.inputs,
                outputs=model_1.outputs,
                name='model_5fold_InceptionV3_GN07mm')
models = [model_1, model_2, model_3]
model_input = Input(shape=(224, 224, 3))
model_outputs = [model(model_input) for model in models]
#ensemble_output = Average()(model_outputs)
ensemble_output = WeightedAverageLayer(0.1, 0.575, 0.325)(model_outputs)
ensemble_model= Model(inputs=model_input, outputs=ensemble_output, name='ensemble')

In [16]:
from keras.models import load_model
import cv2
import numpy as np


class_names = ['early_leaf_spot_1','early_rust_1','healthy_leaf_1',
              'late_leaf_spot_1','nutrition_deficiency_1','rust_1'] # fill the rest

#model = load_model(root_path+"model_5fold_DNMv2_10575325.h5")

ensemble_model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

img = cv2.imread(test_path+"dr_0_25.jpg")
print(img)
img = cv2.resize(img,(224,224))
img = np.reshape(img,[1,224,224,3])

classes = np.argmax(ensemble_model.predict(img), axis = -1)

print(classes)

names = [class_names[i] for i in classes]

print(names)

[[[ 34  61  45]
  [ 34  61  45]
  [ 34  61  45]
  ...
  [152 211 207]
  [156 210 205]
  [159 211 204]]

 [[ 36  63  47]
  [ 36  63  47]
  [ 36  63  47]
  ...
  [152 211 207]
  [156 210 205]
  [159 211 204]]

 [[ 38  66  47]
  [ 38  66  47]
  [ 38  66  47]
  ...
  [154 211 208]
  [158 209 205]
  [160 211 204]]

 ...

 [[ 32  51  58]
  [ 33  53  58]
  [ 36  57  59]
  ...
  [100 164 164]
  [100 164 164]
  [101 165 165]]

 [[ 32  51  58]
  [ 34  54  59]
  [ 36  57  59]
  ...
  [ 96 162 161]
  [ 95 161 160]
  [ 95 161 160]]

 [[ 32  51  58]
  [ 34  54  59]
  [ 36  57  59]
  ...
  [ 99 165 164]
  [101 167 166]
  [101 167 166]]]
1/1 [==============================] - 3s 3s/step
[5]
['rust_1']


In [17]:
from tensorflow.keras.models import save_model

In [18]:
#MODEL_FILENAMEs=root_path+"model_5fold_DNMv2_10775125.h5"
#ensemble_model.save(MODEL_FILENAMEs)

In [19]:
import tensorflow as tf

model = tf.keras.models.load_model(root_path+"model_5fold_ResNet50V2_GN07.h5")
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
open("model_5fold_ResNet50V2_GN07.tflite", "wb").write(tflite_model)

94150760